# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. 

**Цель исследования** - 
Разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 
Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Ход исследования**-
Данные получаем из файла data.csv. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.

Проверим данные на ошибки и оценим их влияние на исследование. На этапе предобработки поищем возможность исправить самые критичные ошибки данных.


Таким образом, исследование пройдёт в три основных этапа:

1. Обзор данных
2. Предобработка данных.
3. Ответы на вопросы



### Шаг 1. Обзор данных

In [1]:
import warnings #импортируем управление предупреждениями
warnings.filterwarnings('ignore') #просим игнорировать предупреждения
import pandas as pd #импортируем библиотеку pandas
data = pd.read_csv('/datasets/data.csv') #чтение файла с данными и сохранение в переменной data
data.head(10) #получение первых 10 строк таблицы data

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/data.csv'

In [ ]:
data.info() #получение общей информации о данных в таблице data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Видим, что таблица состоит из 21525 строк и 12 столбцов, также замечаем пропущеные значения в двух столбцах days_employed и total_income. 

Еще присутствует аномалия данных в колонке days_employed. Указаны отрицательные значения, судя по документации, данный столбец отображает трудовой стаж в днях. Выясним по какой причине образовались данные значения.

Согласно документации к данным:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

В названиях колонок нарушений стиля не обнаружено.

Выводы

В каждой строке таблицы — информация о платёжеспособности клиентов. Часть колонок описывают общую информацию о клиентах: пол, возраст, количество детей в семье, доход, общий трудовой стаж, уровень образования, семейное положение. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. 

Чтобы двигаться дальше, нужно устранить проблемы в данных: преобразовать значения в колонке days_employed, заполнить пропуски в колонках days_employed и total_income.


### Шаг 2.1 Заполнение пропусков

Первым делом обработаем пустые значения в столбце total_income.
Методом isna() найдём все строки с пропусками в столбце total_income и просмотрим первые пять.


In [ ]:
data[data['total_income'].isna()].head() #Ищем пропущенные значения в столбце total_income

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Находим пустые значения типа NaN.  

Проверим, какую долю составляют пропущенные значения в каждом из столбцов с пропусками.


In [ ]:

round(data.isna().sum()*100/len(data),2) #доля пропущенных значений


children             0.0
days_employed       10.1
dob_years            0.0
education            0.0
education_id         0.0
family_status        0.0
family_status_id     0.0
gender               0.0
income_type          0.0
debt                 0.0
total_income        10.1
purpose              0.0
dtype: float64

Пропуски в столбце total_income совпадают с пропусками в столбце days_employed, из чего можем сделать вывод, что отсутствие данных не случайно: в данной ситуации можем предположить, что при внесении данных, у клиентов без ежемесячного дохода по ошибке не заполнялся и трудовой стаж.

Так как значения в столбце total_income - количественные переменные - заполним пропуски характерными значениями. 

Заполним пропущенные значения медианным значением по столбцу, так как доходы клиентов могут сильно отличаться. Среднее значение, в данном случае, будет некорректно характеризовать данные.

In [ ]:
total_income_median = data['total_income'].median() #считаем медиану для столбца total_income
print(total_income_median)

145017.93753253992


In [ ]:
data['total_income'] = data['total_income'].fillna(value=total_income_median) #заполняем пропуски медианным значением
data.isna().sum() #проверяем, заменились ли пропуски 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

Теперь рассмотрим и обработаем значения в столбце days_employed. На начальном этапе знакомства с данными мы сразу заметили отрицательные значения в данном столбце.
Можем предположить, что отрицательные значения возникли вследствие сбоя при загрузке данных, но стоит отметить, что в таблице присутствуют и положительные данные, но невозможно большие.

При первом знакомстве с данными можно предположить, что положительные числа указаны преимущественно для пенсионеров, давайте проверим.

In [ ]:
data.loc[data['days_employed'] > 0, 'income_type']

4        пенсионер
18       пенсионер
24       пенсионер
25       пенсионер
30       пенсионер
           ...    
21505    пенсионер
21508    пенсионер
21509    пенсионер
21518    пенсионер
21521    пенсионер
Name: income_type, Length: 3445, dtype: object

Появляется предположение, что данные некорректно заполняются для клиентов, которые в данный момент времени не работают.
Для этого проверим трудовой стаж для безработных клиентов. 

In [ ]:
data.loc[data['income_type'] =='безработный', 'days_employed']

3133     337524.466835
14798    395302.838654
Name: days_employed, dtype: float64

Предположение подтвердилось, но, так как в нашем исследовании мы не будем использовать информацию о трудовом стаже, исправлять данные значения не целесообразно. На практике точные причины появления этих аномальных значений нужно уточнить у разработчиков.

Обработаем пропущенные значения в столбце days_employed.

In [ ]:
data[data['days_employed'].isna()].head() #Ищем пропущенные значения в столбце days_employed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,145017.937533,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,145017.937533,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,145017.937533,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,145017.937533,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,145017.937533,сыграть свадьбу



Уберем минус в значениях для столбца days_employed. Для этого применим функцию abs, которая возвращает абсолютную величину числа.

In [ ]:
data['days_employed'] = abs(data['days_employed']) #возвращаем абсолютную величину числа

Значения в столбце обновили, теперь заменим пропуски медианным значением по столбцу days_employed.

In [ ]:
days_employed_median = data['days_employed'].median() #считаем медиану для столбца total_income
print(days_employed_median)

2194.220566878695


In [ ]:
data['days_employed'] = data['days_employed'].fillna(value=days_employed_median) #заполняем пропуски медианным значением
data.isna().sum() #проверяем, заменились ли пропуски 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски заполнили, давайте также проверим значения в столбце children, который непосредственно будет учавствовать в исследовании. 

In [ ]:
data['children'].value_counts() #проверим уникальные значения в столбце children

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Замечаем аномальные значения: количество детей -1.
Предположим, что данные записаны с ошибкой, заменим -1 на 1.
Также присутствует значение 20, что маловероятно для 76 клиентов. Но тут мы точно не знаем, имелось в виду 2 или 0, поэтому заменим на медианное значение.

In [ ]:
data['children'] = abs(data['children']) #возвращаем абсолютную величину числа
children_median = data.loc[data.loc[:, 'children'] != 20]['children'].median() #считаем медиану для столбца children
data['children'] = data['children'].replace(20, children_median) #менеям значение 20 на медианное
data['children'] = data['children'].astype('int') #приводим данные в столбце children до целочисленных
data['children'].value_counts() #проверяем результат

0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Теперь проверим данные в столбцах gender и dob_years.

In [ ]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [ ]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Обнаружили одно значение XNA в столбце gender, в данном случае не можем точно предположить, что значение было указано ошибочно, возможно, какой-то клиент целенаправленно не заполнил пол. Так как значения столбца gender не будут учавствовать в нашем исследовании, удалять XNA не будем. 

Также выявили аномальное значение в стобце dob_years, целых 101 клиентов с возрастом 0. В этой ситуации не можем однозначно сказать в чем причина данной аномалии, возможно, при внесении данных сотрудник опечатался и не добавил цифру перед нулем, либо на каком-то этапе произошел сбой. В любом случае эту информацию необходимо уточнить у разработчиков. 

Итак, пропуски и аномальные значения исправлены, перейдем к изменению типов данных и удалению дубликатов. 

### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце total_income на целочисленный с помощью метода astype().

In [ ]:
data['total_income'] = data['total_income'].astype('int')

### Шаг 2.4. Удаление дубликатов.

In [ ]:
data.duplicated().sum() #проверяем наличие явных дубликатов


54

Видим, что в наших данных есть 54 явных дубликата, можем сразу их удалить 

In [ ]:
data = data.drop_duplicates().reset_index(drop=True) #удаляем явные дубликаты
data.duplicated().sum() #проверяем наличие дубликатов после удаления 

0

Проверим столбец education методом value_counts(), чтобы проанализирать столбец, выбрать каждое уникальное значение и подсчитать частоту его встречаемости в списке.

In [ ]:
data['education'].value_counts() #применяем метод value_counts() к столбцу education

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Глядя на таблицу видим, что в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведим их к одному регистру.

In [ ]:
data['education'] = data['education'].str.lower() #приводим столбец education к нижнему регистру
data['education'].value_counts() # повторно применяем метод value_counts() к столбцу education  

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Явные дубликаты в нашем датасете удалены, неявные дубликаты в столбце education обработаны - приведены к единому нижнему регистру
Для работы с дубликатами использовали метод value_counts() для их обнаружения, и метод str.lower() для исправления. 
Дубликаты могли появиться по нескольким причинам. Например, сбой при загрузке данных, либо вследствие проявления человеческого фактора: при внесении данных несколько сотрудников могли по-разному записывать информацию в колонку. 

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

На данном этапе мы имеем два столбца education_id и education, первый из которых отображает уникальное значение столбца  education. 
Создадим отдельный словарь, где названию категории будет соответствовать номер. И в будущих таблицах будем обращаться уже не к длинной строке, а к её числовому обозначению.

In [ ]:
data_log = data[['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']]
data_log.head(10) #создаем и выводим первые 10 строк нашего нового датафрема


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


In [ ]:
data_education_id = data[['education_id', 'education']] #создаем дополнительный датафрейм-словарь для education
data_education_id.head(10)

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
5,0,высшее
6,0,высшее
7,1,среднее
8,0,высшее
9,1,среднее


In [ ]:
data_education_id = data_education_id.drop_duplicates().reset_index(drop=True) #удаляем дубликаты в data_education_id
data_education_id.sort_values('education_id')

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Проделаем тоже самое со столбцами family_status_id и family_status

In [ ]:
data_family_status = data[['family_status_id', 'family_status']] #создаем дополнительный датафрейм-словарь для family_status
data_family_status.head(10)

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
5,1,гражданский брак
6,0,женат / замужем
7,0,женат / замужем
8,1,гражданский брак
9,0,женат / замужем


In [ ]:
data_family_status = data_family_status.drop_duplicates().reset_index(drop=True) #удаляем дубликаты в data_family_status
data_family_status.sort_values('family_status_id')

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Готово, теперь с нашими данными будет удобнее работать. 

### Шаг 2.6. Категоризация дохода.

Для удобства анализа дохода клиентов создадим категоризацию и разделим по диапазонам:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

И создадим отдельный столбец total_income_category.

In [ ]:
def total_income_group(number): #делаем функцию, которая распределит доходы клиентов по категориям
    if 0<number<= 30000:
        return 'E'
    if 30001<=number<=50000:
        return 'D'
    if 50001<=number<=200000:
        return 'C'
    if 200001<=number<=1000000:
        return 'B'
    if 1000001<=number<=2147483647:
        return 'A'

In [ ]:
data_log['total_income_category'] = data_log['total_income'].apply(total_income_group) #добаляем новый столбец с результатами работы функции
data_log.head(15)


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории: 'операции с автомобилем', 'операции с недвижимостью', 'проведение свадьбы', 'получение образования'.

In [ ]:
data['purpose'].value_counts() #Проверяем уникальные значения в столбце purpose

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Полученные цели распределим на четыре основные категории с помощью функции. 

In [ ]:
def purpose_group(name): #делаем функцию, которая распределит цели клиентов по категориям
    if 'автомоб' in name:
        return 'операции с автомобилем'
    if 'недвиж' in name:
        return 'операции с недвижимостью'
    if 'жил' in name:
        return 'операции с недвижимостью'
    if 'свадьб' in name:
        return 'проведение свадьбы'
    if 'образов' in name:
        return 'получение образования'
   

In [ ]:
data_log['purpose_category'] = data_log['purpose'].apply(purpose_group) #добаляем новый столбец с результатами работы функции
data_log.head(15)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


Получили итоговую таблицу для анализа.

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для анализа будем использовать метод pivot_table.

In [ ]:
data_pivot_children = data_log.pivot_table(index = 'children', values= 'debt')
data_pivot_children


,debt
children,
0,0.075513
1,0.091639
2,0.094542
3,0.081818
4,0.097561
5,0.000000


###### Вывод 1
Исходя из полученной информации мы видим, что клиенты с четырьмя детьми реже платят в срок, а у клиентов вообще без детей проблем с погашением кредита меньше.
Также можем обратить внимание, что информация по клиентам с пятью детьми выводится некорректно, поэтому мы не можем опираться на полученную информацию. Для более глубокого анализа нужно больше данных по клиентам с пятью детьми. 


##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для анализа будем использовать метод pivot_table.

In [ ]:
data_pivot = data_log.pivot_table(index = 'family_status_id', values= 'debt')
data_pivot

,debt
family_status_id,
0,0.075421
1,0.093202
2,0.065693
3,0.071130
4,0.097509


###### Вывод 2
Смотря на таблицу мы видим, что вдовцы и разведенные выплачивают кредиты лучше, чем те кто никогда не бывал в браке и клиенты в гражданском браке.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для анализа будем использовать метод pivot_table.

In [ ]:
data_pivot_income = data_log.pivot_table(index = 'total_income_category', values= 'debt')
data_pivot_income

,debt
total_income_category,
A,0.080000
B,0.070621
C,0.084825
D,0.060000
E,0.090909


###### Вывод 3
По данным можем сказать, что клиенты с уровнем дохода до 30 000 имеют больше проблем с возвратом кредита (Е), а клиенты с доходом от 30 001 и до 50 000 (D) погашают задолженность значительно лучше. 

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Для анализа будем использовать метод pivot_table.

In [ ]:
data_pivot_purpose = data_log.pivot_table(index = 'purpose_category', values= 'debt')
data_pivot_purpose

,debt
purpose_category,
операции с автомобилем,0.093547
операции с недвижимостью,0.072314
получение образования,0.092177
проведение свадьбы,0.079657


###### Вывод 4
Глядя на категории, мы видим, что те клиенты, которые берут кредит для операций с автомобилем и получения образования, испытывают больше трудностей с погашением задолженности, чем те, кто берут кредит для операций с недвижимостью и проведения свадебных церемоний.

## Общий вывод:

В ходе данного иследования мы выяснили, что семейное положение и количество детей влияет на факт погошения кредита в срок, так как клиенты овдовевшие и не имеющие детей лучше справляются с погашением задолженостей. Также мы увидели, что клиенты с доходом от 30 до 50 тысяч и те, кто берет кредит на операции по недвижимости, хорошо справляются с погашением задолженности.